In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten

In [2]:
import tensorflow
device_name = tensorflow.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
def main():
    os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private' 

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2,horizontal_flip=True,vertical_flip=False)

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
x_train = train_datagen.flow_from_directory("data/train",target_size=(64,64),class_mode='categorical',batch_size=32)

Found 15341 images belonging to 6 classes.


In [7]:
x_test = test_datagen.flow_from_directory("data/test",target_size=(64,64),class_mode='categorical',batch_size=32)

Found 6825 images belonging to 6 classes.


In [8]:
x_train.class_indices

{'Left Bundle Branch Block': 0,
 'Normal': 1,
 'Premature Atrial Contraction': 2,
 'Premature Ventricular Contractions': 3,
 'Right Bundle Branch Block': 4,
 'Ventricular Fibrillation': 5}

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten,Dropout

In [10]:
model=Sequential()
model.add(Convolution2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=128,kernel_initializer="random_uniform",activation="relu"))
model.add(Dense(units=128,kernel_initializer="random_uniform",activation="relu"))
model.add(Dense(units=128,kernel_initializer="random_uniform",activation="relu"))
model.add(Dense(units=128,kernel_initializer="random_uniform",activation="relu"))
model.add(Dense(units=128,kernel_initializer="random_uniform",activation="relu"))
model.add(Dense(units = 6, kernel_initializer="random_uniform", activation = 'softmax'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1

In [12]:
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
len(x_train)

480

In [13]:
epoch=10
history = model.fit(x_train,steps_per_epoch=len(x_train),validation_data=x_test,validation_steps=len(x_test),epochs=epoch)

Epoch 1/10
480/480 [==============================] - 170s 332ms/step - loss: 1.4493 - accuracy: 0.4780 - val_loss: 1.6764 - val_accuracy: 0.3193
Epoch 2/10
480/480 [==============================] - 24s 50ms/step - loss: 1.1341 - accuracy: 0.5727 - val_loss: 1.3422 - val_accuracy: 0.5194
Epoch 3/10
480/480 [==============================] - 25s 52ms/step - loss: 0.5962 - accuracy: 0.7865 - val_loss: 1.1789 - val_accuracy: 0.6796
Epoch 4/10
480/480 [==============================] - 24s 50ms/step - loss: 0.3330 - accuracy: 0.8887 - val_loss: 0.7553 - val_accuracy: 0.7952
Epoch 5/10
480/480 [==============================] - 24s 50ms/step - loss: 0.2249 - accuracy: 0.9245 - val_loss: 0.5240 - val_accuracy: 0.8503
Epoch 6/10
480/480 [==============================] - 24s 50ms/step - loss: 0.1742 - accuracy: 0.9462 - val_loss: 0.5899 - val_accuracy: 0.8492
Epoch 7/10
480/480 [==============================] - 25s 52ms/step - loss: 0.1329 - accuracy: 0.9597 - val_loss: 0.4521 - val_accurac

In [14]:
model.save('ECG.h5')

In [15]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np

In [16]:
model = load_model('ECG.h5')
img=image.load_img("data/test/Premature Ventricular Contractions/fig_5656.png",target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
print(x.shape)
y=np.argmax(model.predict(x),axis=1)
index=['Left Bundle Branch Block', 'Normal', 'Premature Atrial Contraction', 'Premature Ventricular Contractions', 'Right Bundle Branch Block','Ventricular Fibrillation']
index[y[0]]

(1, 64, 64, 3)


'Normal'